In [2]:
import src.data.get_data as gd
import pandas as pd
import numpy as np
import config

df = gd.main(
    url=config.URL,
    sec_wait=config.SEC_WAITING,
    data_file_path=config.DATA_FILES
)

Connecting to Selenium
Successfully connected to Selenium
Starting data collection
Collected 0 pages
Saving data
Collected 100 pages
Collected 200 pages
Collected 300 pages
Collected 400 pages
Collected 500 pages
Saving data
Collected 600 pages
Collected 700 pages
Collected 800 pages
Collected 900 pages
Collected 1000 pages
Saving data
Collected 1100 pages
Collected 1200 pages
Collected 1300 pages
Collected 1400 pages
Collected 1500 pages
Saving data
Collected 1600 pages
Collected 1700 pages
Collected 1800 pages
Collected 1900 pages
Collected 2000 pages
Saving data
Collected 2100 pages
Collected 2200 pages
Collected 2300 pages
Collected 2400 pages
Collected 2500 pages
Saving data
Collected 2600 pages
Collected 2700 pages
Collected 2800 pages
Finished data collection


In [6]:
df = df.replace({
    'Neg': -1,
    np.nan: 0,
    'Pos': 1 
})

In [11]:
df.groupby(['CITY', 'SITE']).size().sort_values(ascending=False).head(50)

CITY                  SITE                               
Vancouver             Get Your Drugs Tested                  35380
                      Insite                                  6283
                      Overdose Prevention Society             3254
                      Molson OPS                              1986
Nelson                ANKORS (Nelson)                         1571
Surrey                SafePoint                               1366
Kamloops              ASK Wellness (433 Tranquille)            886
Nanaimo               CMHA                                     662
Cranbrook             ANKORS (Cranbrook)                       539
New Westminster       Purpose Society                          454
Prince George         POUNDS                                   295
Vernon                Vernon MHSU                              270
Kelowna               Living Positive Resource Centre          259
Vernon                Cammy LaFleur Outreach (Vernon)          252
Vanc

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
from os.path import join
import time
import os
import datetime
import pickle
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import threading
import queue

In [3]:
class Pages(queue.Queue):
    def __contains__(self, item):
        with self.mutex:
            return item in self.queue

    def __len__(self):
        return len(self.queue)

In [9]:
class Crawler():
    def __init__(self, Queue, **kwargs):
        # sourcery skip: remove-pass-body
        self.queue = Queue
        self.url = kwargs['url']
        self.seconds_to_wait = kwargs['sec_wait']
        self.data_file_path = kwargs['data_file_path']

    def teardown_method(self):
        self.driver.quit()

    def connect_multi(self):
        print("Connecting thread to Selenium")
        self.driver = webdriver.Chrome()
        self.driver.get(self.url)
        self.driver.set_window_size(1280, 680)
        time.sleep(self.seconds_to_wait)
        self.driver.switch_to.frame(0)
        self.driver.find_element(By.CSS_SELECTOR, '#tab_container > li:nth-child(3) > a').click()
        print("Successfully connected thread to Selenium")

    def make_df(self):
        page_source = self.driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        table = soup.find('table')
        return pd.read_html(str(table))[0]

    def collect_data(self, page=0):
        print("Starting data collection")
        self.driver.find_element(By.CLASS_NAME, "current-page").click()
        self.driver.find_element(By.CLASS_NAME, "current-page").send_keys(str(page))
        self.driver.find_element(By.CLASS_NAME, "current-page").send_keys(Keys.ENTER)
        df = self.make_df()
        print("Finished data collection")
        return df

    def run(self):
        self.connect_multi()
        while True:
            page = self.queue.get()
            try:
                if page is None:
                    break
                df = self.collect_data(page=page)
            except Exception as e:
                print("Encountered exception: ", e)
            self.queue.task_done()
        self.teardown_method()
        return df

In [5]:
def get_page_num(driver, url, sec_wait):
    print("Connecting to Selenium")
    driver.get(url)
    driver.set_window_size(1280, 680)
    time.sleep(sec_wait)
    driver.switch_to.frame(0)
    driver.find_element(By.XPATH, '//*[@id="tab_container"]/li[3]/a').click()
    pages = driver.find_element(By.XPATH,
        '/html/body/div/div/div[3]/div/div/div[3]/div/div/div[2]/div/div/div/div[3]/div/div[2]'
    ).text
    print(f"Found {pages} pages")
    driver.quit()
    return int(pages)

In [7]:
Q = Pages()
driver = webdriver.Chrome()
pages = get_page_num(driver, "https://drugcheckingbc.ca/dashboard/", 40)
Q.put(pages)

Connecting to Selenium
Found 3004 pages
Connecting thread to Selenium
Connecting thread to Selenium


Exception in thread Thread-3 (run):
Traceback (most recent call last):
  File "c:\Users\broug\mambaforge\envs\drug-testing\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\broug\mambaforge\envs\drug-testing\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\broug\AppData\Local\Temp\ipykernel_2836\1043009932.py", line 38, in run
  File "C:\Users\broug\AppData\Local\Temp\ipykernel_2836\1043009932.py", line 18, in connect_multi
  File "c:\Users\broug\mambaforge\envs\drug-testing\Lib\site-packages\selenium\webdriver\remote\switch_to.py", line 92, in frame
    self._driver.execute(Command.SWITCH_TO_FRAME, {"id": frame_reference})
  File "c:\Users\broug\mambaforge\envs\drug-testing\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 440, in execute
    self.error_handler.check_response(response)
  File "c:\Users\broug\mambaforge\envs\drug-testing\Lib\site-packages\selenium\webdriver\remote\errorhandl

Successfully connected thread to Selenium
Starting data collection
Encountered exception:  Message: element not interactable
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0033F243]
	(No symbol) [0x002C7FD1]
	(No symbol) [0x001BCF10]
	(No symbol) [0x001ED003]
	(No symbol) [0x001E32A6]
	(No symbol) [0x0020858C]
	(No symbol) [0x001E2BFF]
	(No symbol) [0x00208804]
	(No symbol) [0x0021C9EB]
	(No symbol) [0x00208386]
	(No symbol) [0x001E163C]
	(No symbol) [0x001E269D]
	GetHandleVerifier [0x005D9A22+2655074]
	GetHandleVerifier [0x005CCA24+2601828]
	GetHandleVerifier [0x003E8C0A+619850]
	GetHandleVerifier [0x003E7830+614768]
	(No symbol) [0x002D05FC]
	(No symbol) [0x002D5968]
	(No symbol) [0x002D5A55]
	(No symbol) [0x002E051B]
	BaseThreadInitThunk [0x76A1FA29+25]
	RtlGetAppContainerNamedObjectPath [0x770A7A7E+286]
	RtlGetAppContainerNamedObjectPath [0x770A7A4E+238]



In [10]:
crawlers =[Crawler(Q, url="https://drugcheckingbc.ca/dashboard/", sec_wait=40, data_file_path="../../data/") for _ in range(2)]
for crawler in crawlers:
    t = threading.Thread(target=crawler.run)
    t.daemon = True
    t.start()
Q.join()

Connecting thread to Selenium
Connecting thread to Selenium


Exception in thread Thread-8 (run):
Traceback (most recent call last):
  File "c:\Users\broug\mambaforge\envs\drug-testing\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\broug\mambaforge\envs\drug-testing\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\broug\AppData\Local\Temp\ipykernel_2836\2483615087.py", line 38, in run
  File "C:\Users\broug\AppData\Local\Temp\ipykernel_2836\2483615087.py", line 19, in connect_multi
  File "c:\Users\broug\mambaforge\envs\drug-testing\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 830, in find_element
    return self.execute(Command.FIND_ELEMENT, {"using": by, "value": value})["value"]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\broug\mambaforge\envs\drug-testing\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 440, in execute
    self.error_handler.check_response(response)
  File "c:

Successfully connected thread to Selenium


In [4]:
from .. import config

ImportError: attempted relative import with no known parent package